# Scikit-learn example: Sample pipeline for text feature extraction and evaluation

Exemple original:<br>https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py

Dans ce notebook, on teste un exemple de scikit-learn sur notre jeu de données. Nous allons traiter le texte contenu dans la colonne *Description* pour prédire le prix par nuit d'un logement Airbnb par une régression.

## Analyse et preprocessing du jeu de données

### Import des données

On importe le jeu de données et on sélectionne uniquement les colonnes:
- *Description*: colonne qui décrit les caractéristiques d'un logement Airbnb donné
- *prix_nuitee*: prix par nuit du logement

In [1]:
import pandas as pd

In [14]:
# TO DO: retrieve dataset from OpenML using its ID !
url = 'https://www.data.gouv.fr/fr/datasets/r/123e1c18-37e0-4147-ad65-768320387800'
data = pd.read_csv(url)

In [17]:
data = data[data['Description'].notna()]

In [19]:
X = data['Description']
y = data['prix_nuitee'].values

### Détéction de la langue

Voici à quoi ressemble le premier élément de la colonne *Description*:

In [5]:
print(X[0][:150])

Le logement

We speak english, wir sprechen deutsch, hablamos espanol ! 

En plein centre ville de Bordeaux, dans un cadre familial (enfants et animau


Plusieurs langues apparaissent donc parfois dans la même description, nous allons donc nous restraindre au français:

In [6]:
from langdetect import detect

In [7]:
def select_french_text(text):
    """This function returns a text only containing French sentences"""
    punc = '''´!()-[]{};:'"\,<>./?@#$%^&*_~=•'''
    for element in text:
        if element in punc:
            text = text.replace(element,"")  #langdetect fails on lines only containing punctuation
    text_lines = [line for line in text.splitlines() if (not line.isspace()) if (len(line)!=0)] #langdetect fails on empty lines
    for line in text_lines:
        try:
            if detect(line)!='fr':
                text = text.replace(line,"")       
        except Exception as e:
            print(e)
            print(line)
    return text

In [8]:
X = X.apply(select_french_text)

On peut maintenant séparer le jeu de données en jeu de données d'entraînement et de test:

In [11]:
from sklearn.model_selection import train_test_split 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Regression pipeline

On implémente ici la pipeline proposée dans l'exemple scikit-learn, dans notre cas en faisant une regression au lieu d'une classification:

In [20]:
# Original code:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Mathieu Blondel <mathieu@mblondel.org>
# License: BSD 3 clause
##############################
# Adapted by: Giulia Santarsieri <giulia.santarsieri@data.gouv.fr> for DGML

from pprint import pprint
from time import time
import logging

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')



# #############################################################################
# Define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('reg', SGDRegressor()),
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'reg__alpha': (0.00001, 0.000001),
    'reg__penalty': ('l2', 'elasticnet'),
    'reg__max_iter':(2000,)
}

if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Automatically created module for IPython interactive environment
Performing grid search...
pipeline: ['vect', 'tfidf', 'reg']
parameters:
{'reg__alpha': (1e-05, 1e-06),
 'reg__max_iter': (2000,),
 'reg__penalty': ('l2', 'elasticnet'),
 'tfidf__norm': ('l1', 'l2'),
 'tfidf__use_idf': (True, False),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 96 candidates, totalling 480 fits
done in 1707.753s

Best score: 0.176
Best parameters set:
	reg__alpha: 1e-05
	reg__max_iter: 2000
	reg__penalty: 'l2'
	tfidf__norm: 'l2'
	tfidf__use_idf: True
	vect__max_df: 0.5
	vect__ngram_range: (1, 2)


/home/giulia/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:1225: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


Les résultats ne sont pas satisfaisants (meilleur score égal à 0.176), vous pouvez tester d'autres paramètres dans les étapes de la pipeline ou bien rajouter des étapes à la pipeline pour essayer d'améliorer la performance du modèle!